# Red de entrenamiento
Ammi Beltrán y Fernanda Borja

***

# Import de librerías 

In [2]:
# Libreria de preprocesamiento
import dataprep as prep
# 
import numpy as np
import matplotlib.pyplot as plt
#
import torch
from torch import nn

## Modelo

### Channel Augmenter

### Encoder convolucional

<img src="img/renc.png" alt="Drawing" style="width: 800px;"/>

In [ ]:
class Convolutional_Enc(nn.Module):
    '''
    Encoder convolucional paper 2
    '''
    def __init__(self):
        super(Convolutional_Enc, self).__init__()
        # Reflection pad
        nn.ReflectionPad1d()
        # We begin with 3 convolutional blocks (top to bottom)
        self.conv1 = nn.Sequential(
            # To fit kernel size = 128
            nn.ReflectionPad1d(63,64),
            nn.Conv1d(in_channels = 1, out_channels = 100, kernel_size = 128),
        )
        self.conv2 = nn.Sequential(
            # To fit kernel size = 64
            nn.ReflectionPad1d(31,32),
            nn.Conv1d(in_channels = 1, out_channels = 100, kernel_size = 64),
        )
        self.conv3 = nn.Sequential(
            # To fit kernel size = 16
            nn.ReflectionPad1d(7,8),
            nn.Conv1d(in_channels = 1, out_channels = 50, kernel_size = 16),
        )
        # Repeated block
        self.iterable = nn.Sequential(
            nn.Linear(in_features = 250, out_features = 250),
            nn.ReLU(),
            nn.BatchNorm1d(num_features = 250),
            nn.ReflectionPad1d(31, 32),
            nn.Conv1d(in_channels = 64, out_channels = 250, kernel_size = 64),
        )
        # Outer layer
        self.outer = nn.Sequential(
            nn.ReLU(),
            nn.BatchNorm1d(num_features = 250),
            nn.ReflectionPad1d(31, 32),
            nn.Conv1d(in_channels = 250, out_channels = 4, kernel_size = 64),
        )

    def forward(self, x):
        # Pass trough convolutionals
        xc1 = self.conv1(x)
        xc2 = self.conv2(x)
        xc3 = self.conv3(x)
        # Concatenate
        cat = torch.cat((xc1, xc2, xc3), dim = 1)
        # Iterable phase
        for i in range(4):
            cat = self.iterable(cat)
        # End layer
        end = self.outer(cat)
        #
        return end

### Proyector

<img src="img/proyector.png" alt="Drawing" style="width: 800px;"/>

In [ ]:
class Projector(nn.Module):
    def __init__(self):
        super(Projector, self).__init__()

        # LSTM's bidireccionales
        self.lstm1 = nn.LSTM(
            input_size = 4,
            hidden_size = 256,
            bidirectional = True
        )
        self.lstm2 = nn.LSTM(
            input_size = 4,
            hidden_size = 128,
            bidirectional = True
        )
        self.lstm3 = nn.LSTM(
            input_size = 4,
            hidden_size = 64,
            bidirectional = True
        )
        # Outer Layer
        # Takes first and last output
        self.outer = nn.Sequential(
            nn.Linear(in_features = 4*(64 + 128 + 256), out_features = 128),
            nn.ReLU(),
            nn.Linear(in_features = 128, out_features = 32),
        )
    def forward(self, x):
        # Downsampling
        half = nn.functional.interpolate(x, scale_factor = 0.5)
        quarter = nn.functional.interpolate(x, scale_factor = 0.25)
        # Pass through LSTM
        lstm1 = self.lstm1(x)
        lstm2 = self.lstm2(half)
        lstm3 = self.lstm3(quarter)
        # Get First and Last
        flo1 = lstm1[:, [0, -1], :]
        flo2 = lstm2[:, [0, -1], :]
        flo3 = lstm3[:, [0, -1], :]
        # Concatenate
        cat = torch.cat(flo1, flo2, flo3, dim = 1)
        # Last Layer
        end = self.outer(cat)
        #
        return end

***
### Pretext Task

In [ ]:
class Pretext(nn.Module):
    def __init__(self):
        super(Pretext, self).__init__()
        self.encoder = Convolutional_Enc() 
        self.projector = Projector()
    def forward(self, x):
        first = self.encoder(x)
        end = self.projector(first)
        return end